# Python Built-in Types

We are going to start using spyndex with the simplest objects: `int` and `float`.

Before starting, make sure you have `spyndex` the latest `dask` version. If not, you can upgrade them here:

In [ ]:
!pip install -U spyndex dask

Now, let's start!

First, import `spyndex`:

In [1]:
import spyndex

Now, let's simulate some values for the Near Infrared (`NIR`) and Red (`RED`) reflectances of vegetation with an `int` data type:

In [2]:
NIR = 6723
RED = 1243

If we check the data types for each one of our variables, we will see that they are `int`:

In [7]:
print(f"NIR type: {type(NIR)}, value: {NIR}")
print(f"RED type: {type(RED)}, value: {RED}")

NIR type: <class 'int'>, value: 6723
RED type: <class 'int'>, value: 1243


Now, let's go and compute the `NDVI`! Let's check the attributes of the index:

In [8]:
spyndex.indices.NDVI

NDVI: Normalized Difference Vegetation Index (attributes = ['bands', 'contributor', 'date_of_addition', 'formula', 'long_name', 'reference', 'short_name', 'type'])

From these attributes, we need to know the standard name of the bands so we can compute it:

In [9]:
spyndex.indices.NDVI.bands

('N', 'R')

That's it! We need the `N` and `R` parameters! Let's create a `dictionary` to store them (Psst! This dictionary is useful since we can pass it to the `params` argument in the index computation!):

In [ ]:
parameters = {"N": NIR, "R": RED}

In [14]:
idx = spyndex.computeIndex("NDVI", parameters)

In [15]:
print(f"idx type: {type(idx)}, value: {idx}")

idx type: <class 'float'>, value: 0.6879236756213909


In [21]:
spyndex.indices.SAVI.bands

('L', 'N', 'R')

In [22]:
parameters = {"N": NIR / 10000, "R": RED / 10000, "L": 0.5}

In [27]:
print(f"NIR type: {type(parameters['N'])}, value: {parameters['N']}")
print(f"RED type: {type(parameters['R'])}, value: {parameters['R']}")
print(f"L type: {type(parameters['L'])}, value: {parameters['L']}")

NIR type: <class 'float'>, value: 0.6723
RED type: <class 'float'>, value: 0.1243
L type: <class 'float'>, value: 0.5


In [28]:
idx = spyndex.computeIndex(["NDVI","SAVI"], parameters)

In [29]:
print(f"idx type: {type(idx)}, value: {idx}")

idx type: <class 'list'>, value: [0.687923675621391, 0.6339657565941694]


In [30]:
NIR = [0.634, 0.654, 0.567]
RED = [0.123, 0.156, 0.198]

In [31]:
print(f"NIR type: {type(NIR)}, value: {NIR}")
print(f"RED type: {type(RED)}, value: {RED}")

NIR type: <class 'list'>, value: [0.634, 0.654, 0.567]
RED type: <class 'list'>, value: [0.123, 0.156, 0.198]


In [36]:
parameters = {"N": NIR, "R": RED, "L": 0.5}

In [37]:
idx = spyndex.computeIndex(["NDVI","SAVI"], parameters)

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [38]:
import numpy as np

In [39]:
parameters = {"N": np.array(NIR), "R": np.array(RED), "L": 0.5}

In [40]:
print(f"NIR type: {type(parameters['N'])}, value: {parameters['N']}")
print(f"RED type: {type(parameters['R'])}, value: {parameters['R']}")
print(f"L type: {type(parameters['L'])}, value: {parameters['L']}")

NIR type: <class 'numpy.ndarray'>, value: [0.634 0.654 0.567]
RED type: <class 'numpy.ndarray'>, value: [0.123 0.156 0.198]
L type: <class 'float'>, value: 0.5


In [41]:
idx = spyndex.computeIndex(["NDVI","SAVI"], parameters)

In [42]:
print(f"idx type: {type(idx)}, value: {idx}")

idx type: <class 'numpy.ndarray'>, value: [[0.67503303 0.61481481 0.48235294]
 [0.6097852  0.57022901 0.43754941]]
